# Retrieving intertexts

### Loading needed libraries

In [1]:
import glob
import os
from collections import Counter
from itertools import combinations, product

import numpy as np
np.random.seed(18012023)

import pandas as pd
pd.set_option('display.max_colwidth', 0)

import seaborn as sb

from sklearn.metrics import pairwise_distances
import lxml.etree
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import matplotlib
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.family'] = 'Arial'
from IPython.display import display

In [2]:
fig_dir = '../figures'
if not os.path.isdir(fig_dir):
    os.mkdir(fig_dir)

### Loading data and metadata

In [3]:
meta_df = pd.read_excel('../data/metadata.xlsx')
meta_df = meta_df[meta_df['exclude'] != 'x']
meta_df.sample(10)

,id,title,author,provenance,date_range,genre,subgenre,exclude
204,sinte_kunera,Sinte Kunera,NaN,cdrom-mnl,1400-1500,Epiek,Heiligenleven,NaN
63,florigout_fragm_l,Florigout,NaN,cdrom-mnl,1375-1400,Epiek,Ridder,NaN
111,lippijn,Lippijn,NaN,cdrom-mnl,1400-1420,Dramatiek,NaN,NaN
86,Merlijn-Maerlant,Merlijn,Jacob van Maerlant,cdrom-mnl,1415-1435,Epiek,Arthur,NaN
236,spiegel_historiael__4_velthem__fragm_l,Spiegel historiael (P4),Lodewijk van Velthem,cdrom-mnl,1340-1360,Epiek,Historiografie,NaN
154,ongeidentificeerd_fragment,Ongeïdentificeerd fragment,NaN,cdrom-mnl,1380-1400,Epiek,Didactiek,NaN
215,spiegel_der_sonden_fragm_ge1,Spiegel der sonden,NaN,cdrom-mnl,1300-1400,Epiek,Didactiek,NaN
123,madelgijs_fragm_p,Madelgijs,NaN,cdrom-mnl,1375-1400,Epiek,Karel,NaN
168,renout_van_montalbaen_fragm_be,Renout van Montalbaen,NaN,cdrom-mnl,1340-1360,Epiek,Karel,NaN
242,tien_plaghen,Tien plaghen ende die tien ghebode,NaN,cdrom-mnl,1390-1410,Epiek,Didactiek,NaN


In [4]:
# create pairs of the verses

def get_verse_groups(verses, size=2, intertexts=False):
    for i in range(len(verses) - (size - 1)):
        if not intertexts:
            yield ' / '.join(verses[i : i + size])
        else:
            its = Counter(verses[i : i + size])
            if None in its:
                yield None
            elif len(its) > 1:
                yield 'overlap'
            else:
                yield list(its.keys())[0]

In [5]:
def parse_xml(fn, rm_interpol=False):
    try:
        tree = lxml.etree.parse(fn)
    except OSError:
        print(f'- Could not load {fn}')
        return None
        
    if rm_interpol:
        for interpolation in tree.xpath("//interpolation"):
            interpolation.getparent().remove(interpolation)
        
    for line_node in tree.iterfind('//l'):
        try:
            intertext_id = line_node.attrib['intertext']
        except KeyError:
            intertext_id = None
        
        tokens_ = line_node.attrib['tokens'].split()
        lemmas_ = []
        
        lemma_tags = [l.text for l in line_node.iterfind('.//lemma')]
        pos_tags = [p.text for p in line_node.iterfind('.//pos')]
        
        for lemma, pos in zip(lemma_tags, pos_tags):
            for l, p in zip(lemma.split('+'), pos.split('+')):
                    if p == 'n(prop)':
                        lemmas_.append('n(prop)')
                    else:
                        lemmas_.append(l)
    
        yield tokens_, lemmas_, intertext_id

In [6]:
# create a table with the titles, tokens, lemmas and interetexts for every pair of verses

GROUP_SIZE = 2

titles, tokens, lemmas, intertexts = [], [], [], []

for title, group in tqdm(meta_df.groupby('title')):
    work_tokens, work_lemmas, work_intertexts = [], [], []
    
    for id_ in sorted(group['id']):
        for tok, lem, intertext_id in parse_xml(f'../data/xml/{id_}.xml'):
            work_tokens.append(tok)
            work_lemmas.append(lem)
            work_intertexts.append(intertext_id)
    
    verse_tokens = [' '.join(v) for v in work_tokens]
    verse_lemmas = [' '.join(v) for v in work_lemmas]

    verse_group_tokens = list(get_verse_groups(verse_tokens, size=GROUP_SIZE))
    verse_group_lemmas = list(get_verse_groups(verse_lemmas, size=GROUP_SIZE))
    verse_group_intertexts = list(get_verse_groups(work_intertexts, size=GROUP_SIZE, intertexts=True))

    tokens.extend(verse_group_tokens)
    lemmas.extend(verse_group_lemmas)
    intertexts.extend(verse_group_intertexts)
    titles.extend([title] * len(verse_group_lemmas))

100%|█████████████████████████████████████████| 205/205 [00:10<00:00, 19.90it/s]


In [7]:
df = pd.DataFrame(zip(titles, tokens, lemmas, intertexts), columns=('title', 'tokens', 'lemmas', 'intertext'))

In [8]:
# add the rhyme bigrams to the verses

def tokenizer(text):
    return text.replace(' / ', ' ').lower().strip().split()

def add_rhyme_column(df):
    rhyme_words = []
    for lemmas in df['lemmas']:
        rhymes = []
        for verse in lemmas.split(' / '):
            rhymes.append(verse.strip().split()[-1])
        rhyme_words.append(' '.join(rhymes))
    df['rhyme'] = rhyme_words
    return df

In [9]:
df = add_rhyme_column(df)
df.head()

,title,tokens,lemmas,intertext,rhyme
0,AB recht ende averecht,Aensiet dese vrouwen hoe si gaen / Besiet hoe haer tuten staen,aanzien deze vrouw hoe zij gaan / bezien hoe zij de staan,None,gaan staan
1,AB recht ende averecht,Besiet hoe haer tuten staen / Claer dat si hen blanketten,bezien hoe zij de staan / klaar dat zij zij n(prop),None,staan n(prop)
2,AB recht ende averecht,Claer dat si hen blanketten / Die cleeder soe lanc dat si hen letten,klaar dat zij zij n(prop) / de kleed zo lang dat zij zij letten,None,n(prop) letten
3,AB recht ende averecht,Die cleeder soe lanc dat si hen letten / Ende sleypen hen nae al op die eerde,de kleed zo lang dat zij zij letten / en slapen zij na al op de aarde,None,letten aarde
4,AB recht ende averecht,Ende sleypen hen nae al op die eerde / Fi diere vuylder hoverde,en slapen zij na al op de aarde / fi duur vouwer hovaardij,None,aarde hovaardij


In [10]:
# thresholds

optim_vs = 7000
optim_rw = 0.1693877551020408
optim_th = 0.42202047865566794

In [11]:
vec = TfidfVectorizer(max_features=optim_vs, min_df=2,
                      tokenizer=tokenizer, token_pattern=None).fit(df['lemmas'])

### Retrieving the nearest verses

In [12]:
# retrieve the most similar verses

def nearest_verses(title1, title2, base_df, vectorizer, prop_filt=2, rhyme_weight=.15):
    A = base_df[base_df['title'] == title1]
    B = base_df[base_df['title'] == title2]
    
    AX = vec.transform(A['lemmas']) + rhyme_weight * vec.transform(A['rhyme'])
    BX = vec.transform(B['lemmas']) + rhyme_weight * vec.transform(B['rhyme'])
    
    nearest = []
    for i, ax in enumerate(AX):
        distances = pairwise_distances(ax, BX, metric='cosine').flatten()
        top_i = distances.argsort()[0]
        nearest.append((A.iloc[i]['tokens'], B.iloc[top_i]['tokens'],
                        A.iloc[i]['lemmas'], B.iloc[top_i]['lemmas'],
                        A.iloc[i]['title'], B.iloc[top_i]['title'],
                        A.iloc[i]['intertext'], B.iloc[top_i]['intertext'],
                        distances[top_i]))

    nearest = pd.DataFrame(nearest,
                           columns=['tokens1', 'tokens2', 'lemmas1', 'lemmas2',
                                    'title1', 'title2', 'intertext1', 'intertext2',
                                    'distance'])

    nearest = nearest[nearest['title1'] != nearest['title2']]
    nearest = nearest[
                      (nearest['lemmas1'].str.count('n\(prop\)') < prop_filt) & \
                      (nearest['lemmas1'].str.count('n\(prop\)') < prop_filt) 
                     ]
    return nearest

In [13]:
nearest_df = nearest_verses('Limborch', 'Roman der Lorreinen II',
                            base_df=df, vectorizer=vec, prop_filt=2)
nearest_df = nearest_df.sort_values(by='distance')
nearest_df.to_excel('../figures/limborch_lorreinenII.xlsx')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,intertext1,intertext2,distance
3335,Smargens doe die dach ontspranc / Entie liwerke sanc,Smargens als die dach ontspranc / Ende die lewerke sanc,de morgen toen de dag ontspringen / de en leeuwerik zingen,de morgen als de dag ontspringen / en de leeuwerik zingen,Limborch,Roman der Lorreinen II,None,None,0.031936
6723,Smergens doe die dach ontspranc / Entie liwerke sanc,Smargens als die dach ontspranc / Ende die lewerke sanc,de morgen toen de dag ontspringen / de en leeuwerik zingen,de morgen als de dag ontspringen / en de leeuwerik zingen,Limborch,Roman der Lorreinen II,None,None,0.031936
14997,Smargens doe de dach ontspranc / Entie liwerke sanc,Smargens als die dach ontspranc / Ende die lewerke sanc,de morgen toen de dag ontspringen / de en leeuwerik zingen,de morgen als de dag ontspringen / en de leeuwerik zingen,Limborch,Roman der Lorreinen II,None,None,0.031936
9024,Ende seide here die macht van gode / Moet u behouden in u ere,Die seide gi heren die macht van gode / Moet v behouden in v ere,en zeggen heer de macht van n(prop) / moeten gij behouden in gij een,die zeggen gij heer de macht van n(prop) / moeten gij behouden in gij een,Limborch,Roman der Lorreinen II,None,None,0.034856
2346,Die den keyser goeden dach / Ontboet tierst dat sine sach,Ende teerst datsi den keyser sach / Ontboet si heme goeden dach,de de keizer goed dag / ontbieden eerst het dat zijn zien,en eerst het dat zij de keizer zien / ontbieden zijn hij goed dag,Limborch,Roman der Lorreinen II,None,None,0.039712
17189,Ic ligghe hier al miin leven lanc / Of ic hebs minen wille,Jc en hebbe van hem minen wille / Jc ligge hier al mijn leuen lanc,ik liggen hier al mijn leven lang / of ik hebben mijn wil,ik en hebben van hij mijn wil / ik liggen hier al mijn leven lang,Limborch,Roman der Lorreinen II,None,None,0.058481
4429,Nu es die ridder in sorgen groet / Ende in anxste van der doet,Die nu sijn in anxste groet / Ende in sorgen van der doet,nu zijn de ridder in zorg groot / en in angst van de dood,die nu zijn in angst groot / en in zorg van de dood,Limborch,Roman der Lorreinen II,None,None,0.059772
15115,Nu laet ic van hem bliven / Ende willu vort bescriven,Nu latic hier van hem bliuen / Ende wille vort bescriuen,nu laten ik van hij blijven / en n(prop) voorts beschrijven,nu ik laten hier van zij blijven / en willen voorts beschrijven,Limborch,Roman der Lorreinen II,None,None,0.111505
14175,Gheseit heeft behagedem wale / Hi seide bi Mamet u tale,Huge seide gi segt wale / Mi behaegt wel uwe tale,zeggen hebben behagen wel / hij zeggen bij n(prop) uw taal,n(prop) zeggen gij zeggen wel / ik behagen wel uw taal,Limborch,Roman der Lorreinen II,None,None,0.111800
482,Ende te vertellen alle die zake / Hoe si gheweest hadde te onghemake,Ende verteldem dese sake / Ende hoe otte ware tongemake,en te vertellen al de zaak / hoe zij zijn hebben te ongemak,en vertellen deze zaak / en hoe n(prop) zijn ongemak te,Limborch,Roman der Lorreinen II,None,None,0.112764
